In [63]:
!pip install opencv-python

In [64]:
import cv2
import IPython.display as ipd
from tqdm import tqdm # progress bar
import subprocess

In [65]:
# https://doc.bdd100k.com/
# http://dl.yf.io/bdd100k/video_parts/
# https://www.kaggle.com/datasets/deeplyft/driving-video-subset-50-with-object-tracking
# Video for Object Detection (yt playlist)
# https://youtube.com/playlist?list=PLcQZGj9lFR7y5WikozDSrdk6UCtAnM9mB&si=WBmV5nw6A8q6JFiQ
cap = cv2.VideoCapture('./data/example.mp4')

In [66]:
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
n_frames

2301

In [67]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f'(width, height) ({width}, {height})')

(width, height) (1280, 720)


In [68]:
fps = round(cap.get(cv2.CAP_PROP_FPS), 2)
fps

29.97

In [69]:
ipd.Video('./data/example.mp4', width=700)

In [70]:
def mark_objects(frame, object_detector):
    mask = object_detector.apply(frame)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 150:
            # cv2.drawContours(frame, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
    return img

In [71]:
!rm -r labeled.mp4
!rm -r temp_labeled.mp4

rm: temp_labeled.mp4: No such file or directory


In [72]:
tmp_out_name = 'temp_labeled.mp4'
out = cv2.VideoWriter(tmp_out_name,
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps,
                      (width, height))
object_detector = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=80)
_, img = cap.read()

for frame in tqdm(range(n_frames), total=n_frames):
    ret, img = cap.read()
    if ret == False:
        break
    img = mark_objects(img, object_detector)
    out.write(img)
out.release()
cap.release()

100%|█████████████████████████████████████▉| 2300/2301 [00:21<00:00, 105.47it/s]


In [73]:
# brew install ffmpeg
# https://www.ffmpeg.org/
out_name = 'labeled.mp4'
subprocess.run(
    [
        "ffmpeg",
        "-i",
        tmp_out_name,
        "-crf",
        "18",
        "-preset",
        "veryfast",
        "-vcodec",
        "libx264",
        out_name,
        '-loglevel',
        'quiet'
    ]
)
# ffmpeg -i labeled.mp4 -filter_complex "fps=24, scale=-1:360" -ss 20.0 -t 10 labeled.gif

CompletedProcess(args=['ffmpeg', '-i', 'temp_labeled.mp4', '-crf', '18', '-preset', 'veryfast', '-vcodec', 'libx264', 'labeled.mp4', '-loglevel', 'quiet'], returncode=0)

In [74]:
!rm -r temp_labeled.mp4

In [75]:
ipd.Video(out_name, width=700)